In [1]:
import math
import torch
from diffusion_edf.dist import IgSO3Dist, isotropic_gaussian_so3, isotropic_gaussian_so3_small_angle, isotropic_gaussian_so3_lie_deriv
from diffusion_edf import transforms
import plotly.graph_objs as go

In [2]:
dist = IgSO3Dist()

In [3]:
dtype = torch.float64
device = 'cpu'

eps = 5.
q = dist.sample(eps = eps, N=3, dtype=dtype, device=device)
#q = torch.tensor([[1., 0., 0., 0.]], dtype=dtype, device=device)

prob = isotropic_gaussian_so3(q, eps)
deriv = isotropic_gaussian_so3_lie_deriv(q, eps)
print(f"Anaylitic Lie-derivative: {deriv}")

dt = 0.00001

prob_rot = []
for i in range(3):
    qrot = transforms.quaternion_multiply(q,transforms.axis_angle_to_quaternion(dt * torch.eye(3, device=q.device, dtype=q.dtype)[i]))
    prob_rot.append(isotropic_gaussian_so3(qrot, eps))

prob_rot = torch.stack(prob_rot, dim=-1)
print(f"Numerical Lie-derivative: {(prob_rot - prob[...,None]) / dt}")
print(f"Allclose: {torch.allclose((prob_rot - prob[...,None]) / dt, deriv)}")

Anaylitic Lie-derivative: tensor([[-1.7123e-04,  1.8254e-04,  1.0743e-04],
        [-2.2139e-05,  1.5872e-04, -1.9410e-05],
        [ 1.9109e-04,  6.9446e-05,  3.7320e-05]], dtype=torch.float64)
Numerical Lie-derivative: tensor([[-1.7123e-04,  1.8254e-04,  1.0743e-04],
        [-2.2140e-05,  1.5872e-04, -1.9410e-05],
        [ 1.9109e-04,  6.9446e-05,  3.7320e-05]], dtype=torch.float64)
Allclose: True
